In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
from sklearn.model_selection import train_test_split
import math
from IPython.display import display

In [2]:
np.random.seed(0)


def gen_N(mean, Sigma, size):
    return np.random.multivariate_normal(mean=mean, cov=Sigma, size=size)


def params_cov(X, mu_hat):
    n, p = X.shape
    S = np.zeros((p, p))

    def s(l, j):
        s = 0

        for i in range(n):
            s += (X[i][l] - mu_hat[l]) * (X[i][j] - mu_hat[j])

        return s / (n - 1)

    for l in range(p):
        for j in range(p):
            S[l][j] = s(l, j)

    return S


def mu_hat(X):
    return np.mean(X, axis=0)


def DF_values(a, X):
    n = X.shape[0]
    z = np.empty((n, 1))

    for i in range(n):
        z[i] = np.dot(a, X[i])

    return z


def get_s_z_sqr(a, S):
    p = S.shape[0]
    s = 0

    for l in range(p):
        for j in range(p):
            s += a[l] * S[l][j] * a[j]

    return s


def Mahalanobis_distance(z1_mean, z2_mean, s_z_sqr):
    return ((z1_mean - z2_mean)**2) / s_z_sqr


def displaced_Mahalanobis_distance(n1, n2, D_sqr, p):
    return ((n1 + n2 - p - 3) / (n1 + n2 - 2)) * D_sqr - p * (1 / n1 + 1 / n2)


def F(x):
    return st.norm.cdf(x)


def parse_file(filename):
    arr = []

    with open(filename) as file:
        for line in file:
            arr.append(list(map(int, line.strip().split())))

    return arr


def split_X(X, arr, offset=0):
    p = X.shape[1]
    X1 = np.empty((0, p))
    X2 = np.empty((0, p))

    for i in range(len(X)):
        if arr[i + offset][-1] == 1:
            X1 = np.append(X1, [X[i]], axis=0)
        else:
            X2 = np.append(X2, [X[i]], axis=0)

    return X1, X2


def analyze(X1, X2, test_part_1, test_part_2):
    def display_correspondence_table(z1, z2):
        table = [[z1[z1 >= c].shape[0], z1[z1 < c].shape[0]],
                 [z2[z2 >= c].shape[0], z2[z2 < c].shape[0]]]

        correspondence_table = pd.DataFrame([], columns=['1', '2'])
        correspondence_table.columns.name = 'True↓ \ Pred→'
        correspondence_table.loc['1'] = table[0]
        correspondence_table.loc['2'] = table[1]

        display(correspondence_table)

        return table

    def print_D():
        print('Смещённая оценка расстояния Махаланобиса:')
        print(D_sqr)
        print()

        print('Несмещённая оценка расстояния Махаланобиса')
        print(D_H_sqr)
        print()

    def print_P_table(table):
        P_2_1_table = table[0][1] / (table[0][0] + table[0][1])
        P_1_2_table = table[1][0] / (table[1][0] + table[1][1])

        print('Табличные вероятности ошибочной классификации:')
        print('P(2|1) = ' + str(P_2_1_table))
        print('P(1|2) = ' + str(P_1_2_table))
        print()

    def print_P_hat():
        P_2_1_hat = F((K - D_H_sqr / 2) / math.sqrt(D_H_sqr))
        P_1_2_hat = F((-K - D_H_sqr / 2) / math.sqrt(D_H_sqr))

        print('Теоретические вероятности ошибочной классификации:')
        print('P^(2|1) = ' + str(P_2_1_hat))
        print('P^(1|2) = ' + str(P_1_2_hat))
        print()

    n1, n2, p = X1.shape[0], X2.shape[0], X1.shape[1]
    q1 = n1 / (n1 + n2)
    q2 = n2 / (n1 + n2)

    mu_hat1 = mu_hat(X1)
    mu_hat2 = mu_hat(X2)
    S1 = params_cov(X1, mu_hat1)
    S2 = params_cov(X2, mu_hat2)
    S = ((n1 - 1) * S1 + (n2 - 1) * S2) / (n1 + n2 - 2)

    a = np.linalg.solve(S, mu_hat1 - mu_hat2)
    s_z_sqr = get_s_z_sqr(a, S)

    z1 = DF_values(a, X1)
    z2 = DF_values(a, X2)
    test_part_1_z = DF_values(a, test_part_1)
    test_part_2_z = DF_values(a, test_part_2)
    z1_mean = np.mean(z1)
    z2_mean = np.mean(z2)

    K = math.log(q2 / q1)
    c = (z1_mean + z2_mean) / 2 + K
    D_sqr = Mahalanobis_distance(z1_mean, z2_mean, s_z_sqr)
    D_H_sqr = displaced_Mahalanobis_distance(n1, n2, D_sqr, p)

    print('Обучающая выборка')
    table = display_correspondence_table(z1, z2)
    print_D()
    print_P_table(table)
    print_P_hat()

    print('Тестовая выборка')
    test_table = display_correspondence_table(test_part_1_z, test_part_2_z)
    print_P_table(test_table)

In [3]:
Sigma = np.array([[1, 0, 0], [0, 4, 0], [0, 0, 9]])
mu1 = np.array([-2, -1, 0])
mu2 = np.array([1, 2, 3])
n1 = 200
n2 = 800
test_part_n = 100

X1 = gen_N(mu1, Sigma, n1)
X2 = gen_N(mu2, Sigma, n2)
test_X1 = gen_N(mu1, Sigma, test_part_n)
test_X2 = gen_N(mu2, Sigma, test_part_n)

analyze(X1, X2, test_X1, test_X2)

Обучающая выборка


True↓ \ Pred→,1,2
1,185,15
2,9,791


Смещённая оценка расстояния Махаланобиса:
14.180862036322667

Несмещённая оценка расстояния Махаланобиса
14.105274913932595

Табличные вероятности ошибочной классификации:
P(2|1) = 0.075
P(1|2) = 0.01125

Теоретические вероятности ошибочной классификации:
P^(2|1) = 0.06568364746938081
P^(1|2) = 0.01232106704972486

Тестовая выборка


True↓ \ Pred→,1,2
1,90,10
2,2,98


Табличные вероятности ошибочной классификации:
P(2|1) = 0.1
P(1|2) = 0.02



In [4]:
arr = parse_file('german.data-numeric')

n = 1000
p = 24
n1 = 700
n2 = 300
q1 = n1 / n
q2 = n2 / n
X = np.empty((0, p))

for i in range(n):
    x = np.array(arr[i][:-1])
    X = np.append(X, [x], axis=0)

train_X, test_X = train_test_split(X, shuffle=False)
X1, X2 = split_X(train_X, arr)
test_X1, test_X2 = split_X(test_X, arr, offset=train_X.shape[0])

analyze(X1, X2, test_X1, test_X2)

Обучающая выборка


True↓ \ Pred→,1,2
1,473,54
2,113,110


Смещённая оценка расстояния Махаланобиса:
1.5858874979682476

Несмещённая оценка расстояния Махаланобиса
1.3797191213229203

Табличные вероятности ошибочной классификации:
P(2|1) = 0.10246679316888045
P(1|2) = 0.5067264573991032

Теоретические вероятности ошибочной классификации:
P^(2|1) = 0.09350320052513472
P^(1|2) = 0.5575941912890766

Тестовая выборка


True↓ \ Pred→,1,2
1,157,16
2,35,42


Табличные вероятности ошибочной классификации:
P(2|1) = 0.09248554913294797
P(1|2) = 0.45454545454545453

